In [ ]:
import cv2 
import numpy as np 
import time

In [17]:
# here for cloak detection i am taking blue color
# capturing background without cloak in frame. this will be used to replace blue areas 
cap = cv2.VideoCapture(0)

time.sleep(3)  #allows camera to warm-up

for i in range(30):
    ret,background = cap.read()   #capturing background
    if not ret: 
        print("Failed to capture background")
        cap.release()
        exit()
    
#flip background
background = np.flip(background,axis=1)

In [18]:
# process each frame to detect the blue color and replace it with the background
while cap.isOpened():
    ret,frame = cap.read()
    if not ret:
        exit()
    
    #flip the frame
    frame = np.flip(frame,axis=1)
    
    #converting hsv color space for ease of detection 
    hsv = cv2.cvtColor(frame,cv2.COLOR_BGR2HSV)
    
    #defining the range of blue color in hsv
    lower_blue = np.array([100,150,0])
    upper_blue = np.array([140,255,255])
    
    #creating mask for blue
    mask = cv2.inRange(hsv,lower_blue,upper_blue)
    #removing noise from mask
    mask = cv2.morphologyEx(mask,cv2.MORPH_OPEN,np.ones((3,3),np.uint8))
    mask = cv2.morphologyEx(mask,cv2.MORPH_DILATE,np.ones((3,3),np.uint8))
    
    #creating inverse mask
    mask_inv = cv2.bitwise_not(mask)
    
    #segmenting cloak part
    res1 = cv2.bitwise_and(background,background,mask=mask)
    #segmenting non-cloak part 
    res2 = cv2.bitwise_and(frame,frame,mask=mask_inv)
    
    #combine the results
    final_output = cv2.addWeighted(res1,1,res2,1,0)
    
    cv2.imshow("Invisibility Cloak",final_output)
    
    #break the loop is 'v' is pressed 
    if cv2.waitKey(1) & 0xFF == ord('v'): 
        break
        
# release camera and close all opencv windows 
cap.release()
cv2.destroyAllWindows()

In [ ]:
def nothing(x):
    pass

# open the camera
cap = cv2.VideoCapture(0)

# create a window
cv2.namedWindow('Trackbars')

# create trackbars for color change
cv2.createTrackbar('Lower Hue', 'Trackbars', 100, 179, nothing)
cv2.createTrackbar('Upper Hue', 'Trackbars', 140, 179, nothing)
cv2.createTrackbar('Lower Saturation', 'Trackbars', 150, 255, nothing)
cv2.createTrackbar('Upper Saturation', 'Trackbars', 255, 255, nothing)
cv2.createTrackbar('Lower Value', 'Trackbars', 0, 255, nothing)
cv2.createTrackbar('Upper Value', 'Trackbars', 255, 255, nothing)

while True:
    # capture frame-by-frame
    ret, frame = cap.read()
    if not ret:
        break
    
    # flip the frame
    frame = np.flip(frame, axis=1)
    
    # convert to HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # get current positions of trackbars
    lh = cv2.getTrackbarPos('Lower Hue', 'Trackbars')
    uh = cv2.getTrackbarPos('Upper Hue', 'Trackbars')
    ls = cv2.getTrackbarPos('Lower Saturation', 'Trackbars')
    us = cv2.getTrackbarPos('Upper Saturation', 'Trackbars')
    lv = cv2.getTrackbarPos('Lower Value', 'Trackbars')
    uv = cv2.getTrackbarPos('Upper Value', 'Trackbars')
    
    # create the mask
    lower_blue = np.array([lh, ls, lv])
    upper_blue = np.array([uh, us, uv])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # apply the mask to the frame
    result = cv2.bitwise_and(frame, frame, mask=mask)
    
    # show the original and masked frames
    cv2.imshow('Frame', frame)
    cv2.imshow('Mask', mask)
    cv2.imshow('Result', result)
    
    # break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# release the camera and destroy all windows
cap.release()
cv2.destroyAllWindows()
